In [8]:
import gym
import numpy as np
import gym_map

env = gym.make('map-bro-v0')

AttributeError: module 'gym_map.envs' has no attribute 'broEnv'

In [ ]:
# Example 1:
# Store 3 equally sized files when there's only space for 2 files
# Should try to compress two of the files by 50%

# Set up the problem
hd_size = 2.0
file_array = np.array([[1.0, 1.0], [1.0,1.0], [1.0,1.0]],np.float)
file_num = 3
compress_ratio = 0.5

env.__myinit__(hd_size, file_num, file_array, compress_ratio)

# Set hyperparameters for learning
lr = .9
y = .95
num_episodes = 100

# Run the agent
np.random.seed(1)
Q, rList = toy_agents.delayed_reward_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)

In [ ]:
# Example 2:
# Store one large file and two small files
# Should try to compress two of the files by 50%

# Set up the problem
hd_size = 3.0
file_array = np.array([[2.0,1.0], [1.0,1.0], [1.0,1.0]],np.float)
file_num = 3
compress_ratio = 0.5

env.__myinit__(hd_size, file_num, file_array, compress_ratio)

# Set hyperparameters for learning
lr = .9
y = .95
num_episodes = 100

# Run the agent
np.random.seed(2)
Q, rList = toy_agents.delayed_reward_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)

In [ ]:
# Example 3:
# Tag first file as more important by increasing reward
# Added a custom reward to decrease reward for compressing
# Should try to save important files without compressing

# Set up the problem
hd_size = 2.0
file_array = np.array([[1.0, 2.0], [1.0,1.0], [1.0,1.0]],np.float)
file_num = 3
compress_ratio = 0.5
def reward_function(self, size, reward, action):
	if(action == 0):
		return reward
	elif(action == 1):
		return reward*compress_ratio
	else:
		return 0

env.__myinit__(hd_size, file_num, file_array, compress_ratio, reward_function)

# Set hyperparameters for learning
lr = .9
y = .95
num_episodes = 100

# Run the agent
np.random.seed(1)
Q, rList = toy_agents.delayed_reward_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)

In [ ]:
# Example 4:
# Tag third file as most important to show that greedy algorithm fails

# Set up the problem
hd_size = 2.1
file_array = np.array([[1.0, 1.0], [1.0,1.0], [1.0,2.0]],np.float)
file_num = 3
compress_ratio = 0.5
def reward_function(self, size, reward, action):
	if(action == 0):
		return reward
	elif(action == 1):
		return reward*compress_ratio
	else:
		return 0

env.__myinit__(hd_size, file_num, file_array, compress_ratio)

# Set hyperparameters for learning
lr = .9
y = .95
num_episodes = 100

# Run the agent
np.random.seed(0)
Q, rList = toy_agents.greedy_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)

In [ ]:
# Example 5
# Repeat example 4 with delayed reward to show it can learn from the future

# Set up the problem
hd_size = 2.1
file_array = np.array([[1.0, 1.0], [1.0,1.0], [1.0,2.0]],np.float)
file_num = 3
compress_ratio = 0.5
def reward_function(self, size, reward, action):
	if(action == 0):
		return reward
	elif(action == 1):
		return reward*compress_ratio
	else:
		return 0

env.__myinit__(hd_size, file_num, file_array, compress_ratio, reward_function)

# Set hyperparameters for learning
lr = 1.0
y = .95
num_episodes = 100

# Run the agent
np.random.seed(5)
Q, rList = toy_agents.delayed_reward_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)

In [ ]:
# Example 6
# One large example
# Two high reward files, two large files, and two normal files

# Set up the problem
hd_size = 4.1
file_array = np.array([ [1.0, 2.0], [1.0, 2.0],
                        [2.0, 1.0], [2.0, 1.0],
                        [1.0, 1.0], [1.0, 1.0]],np.float)
file_num = 6
compress_ratio = 0.5
def reward_function(self, size, reward, action):
	if(action == 0):
		return reward
	elif(action == 1):
		return reward*compress_ratio
	else:
		return 0

env.__myinit__(hd_size, file_num, file_array, compress_ratio, reward_function)

# Set hyperparameters for learning
lr = 1.0
y = .95
num_episodes = 100

# Run the agent
#np.random.seed(2)
Q, rList = toy_agents.delayed_reward_agent(env, lr, y, num_episodes)

# Report the results
print("Score over time: " +  str(sum(rList)/num_episodes))
print("Final Q-Table Values")
print(Q)
toy_agents.recommend(file_array,Q)